## Testing OpenAI's new Responses API and Agents SDK

OpenAI announced on March 11th, 2025 new Responses API and Agents SDK (https://openai.com/index/new-tools-for-building-agents/) 

This notebook aims to test these new products

In [91]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

from pydantic import BaseModel, Field
import json

#prev openai version: 1.59.8
#new openai version: 1.66.2

### 1. Responses API

Syntax

In [8]:
MODEL_NAME = "gpt-4o"
instruction = "Write a one-sentence bedtime story about a unicorn."

### ChatCompletions

completion = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": instruction
        }
    ]
)

### Responses API

response = client.responses.create(
    model=MODEL_NAME,
    temperature=0.0,
    input=[
        {
            "role": "user",
            "content": instruction
        }
    ]
)

In [9]:
print(f'ChatCompletion response: \n{completion.choices[0].message.content}')
print('---' * 20)
print(f'Responses response: \n{response.output_text}')

ChatCompletion response: 
In a moonlit meadow, a gentle unicorn named Luna sprinkled stardust with her shimmering horn, lulling the entire forest into a peaceful, enchanted slumber.
------------------------------------------------------------
Responses response: 
Under the shimmering moonlight, Luna the unicorn gently trotted through the enchanted forest, her mane sparkling with stardust, as she whispered dreams of magic and wonder to all the sleeping creatures.


In [18]:
response.output_text == response.output[0].content[0].text

True

Explore new Responses output object

In [37]:
MODEL_NAME = "o3-mini"

system_message = "You are a witty joke writer. You have no political correctness filter. You're number one goal is to tell jokes without being afraid of offending anyone"

topic = "us-canada geopolitical relations"

response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    reasoning={
        "effort": "high",
        #"generate_summary" : "concise" # detailed
        },
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}"
        }
    ]
)

In [38]:
print(response.output_text)

Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!


In [49]:
input_tokens = response.usage.input_tokens
output_tokens = response.usage.output_tokens
reasoning_tokens = response.usage.output_tokens_details.reasoning_tokens
net_output_tokens = output_tokens - reasoning_tokens

print(f'Input tokens: {input_tokens}')
print(f'Reasoning tokens: {reasoning_tokens}')
print(f'Net output tokens: {net_output_tokens}')
print(f'Output tokens: {output_tokens}')

Input tokens: 83
Reasoning tokens: 3264
Net output tokens: 52
Output tokens: 3316


In [51]:
response.reasoning

Reasoning(effort='high', generate_summary=None)

In [52]:
response.output

[ResponseReasoningItem(id='rs_67d1cdee27a08192beb956158fb8a02d04bbb1bb73a141e7', summary=[], type='reasoning', status=None),
 ResponseOutputMessage(id='msg_67d1cdf354348192a050188302163acc04bbb1bb73a141e7', content=[ResponseOutputText(annotations=[], text='Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!', type='output_text')], role='assistant', status='completed', type='message')]

In [55]:
response.output[-1].content[0].text

'Why did the U.S. call off its plans to invade Canada? Because every time a missile was about to launch, Ottawa simply shouted, "Sorry, eh?"—proving that a barrage of polite apologies is the ultimate weapon of mass deterrence!'

Structured output

The direct BaseModel passing is still in beta, so will convert class to JSON schema

https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses&example=structured-data

In [84]:
class Joke(BaseModel):
    """A joke about the user's preferred topic"""

    joke: str = Field(description="A joke about the user's preferred topic")
    explanation: str = Field(description="An explanation of the joke")

    @classmethod
    def get_name(cls):
        return "joke"

In [87]:
structured_format = {
    "format": {
        "type": "json_schema",
        "name": Joke.get_name(),
        "schema": {
            **Joke.model_json_schema(),
            "additionalProperties": False
        },
        "strict": True
    }
}

In [88]:
response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    reasoning={
        "effort": "high",
        #"generate_summary" : "concise" # detailed
        },
    
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}"
        }
    ],
    text=structured_format
)

In [94]:
try:
    response_dict = json.loads(response.output[-1].content[0].text)
except json.JSONDecodeError as e:
    response_dict = response.output[-1].content[0].text

print(response_dict)

{'joke': "At the latest summit, the US declared, 'We need a wall to keep out our problems,' while Canada just smiled and said, 'Sorry, eh? How about we resolve them over some hockey and maple syrup instead?'", 'explanation': 'This joke plays on the classic stereotypes: the US with its bold, sometimes confrontational policies (like building walls), and Canada with its trademark politeness and laid-back charm, suggesting that even geopolitical disputes could be smoothed over with a friendly game of hockey and a taste of maple syrup.'}


Takeaways:

- system_message can be passed as `instructions` parameter or as a separate `system` message in the `input` parameter.
- there is a `generate_summary` parameter that could be used to show concise or detailed reasoning - but does not work yet with any of the reasoning models.
- no direct Structured Output support, but `json_schema` in `text` field works

Web search tool

https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses

Available values for `search_context_size`:

- high: Most comprehensive context, highest cost, slower response.
- medium (default): Balanced context, cost, and latency.
- low: Least context, lowest cost, fastest response, but potentially lower answer quality.

In [ ]:
MODEL_NAME = 'gpt-4o'

response = client.responses.create(
    model=MODEL_NAME,
    temperature=None if 'o3' in MODEL_NAME or 'o1' in MODEL_NAME else 0.0,
    instructions=system_message,
    input=[
        {
            "role": "user",
            "content": f"Given the user's preferred topic, generate a joke. User's topic: {topic}"
        }
    ],
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
)